In [1]:
import pandas as pd
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

### Season 1

In [2]:
# Create the model
m1 = LpProblem(name='S', sense=LpMinimize)

# Define decision variables
var = [[LpVariable(name='{},{}'.format(i,j), lowBound=0, cat='Binary') for j in range(10)] for i in range(10)]
var_flat = [x for tup in var for x in tup]

# Set objective function
m1 += lpSum(var_flat)

# Define constraints - results of each round
# In rounds: index is guys, values is girls
# In truth_booths: key is (guy, girl), value is result
rounds = []
rounds.append( {'pairs': [3, 5, 7, 8, 0, 4, 2, 1, 9, 6], 'num_matches': 2} ) # round 1
rounds.append( {'pairs': [5, 7, 9, 2, 6, 8, 4, 3, 0, 1], 'num_matches': 4} ) # round 2
rounds.append( {'pairs': [5, 7, 4, 6, 2, 8, 0, 1, 9, 3], 'num_matches': 2} ) # round 3
rounds.append( {'pairs': [7, 1, 0, 6, 8, 2, 3, 5, 9, 4], 'num_matches': 2} ) # round 4
rounds.append( {'pairs': [5, 8, 7, 2, 6, 1, 4, 3, 9, 0], 'num_matches': 5} ) # round 5
rounds.append( {'pairs': [5, 0, 7, 1, 6, 3, 4, 8, 9, 2], 'num_matches': 5} ) # round 6
rounds.append( {'pairs': [5, 8, 7, 4, 6, 2, 1, 3, 9, 0], 'num_matches': 7} ) # round 7
truth_booths = {(0,0): 0, (6,2): 0, (4,3): 0, (8,0): 0, (5,8): 0, 
                (8,9): 1, (4,6): 1, (1,7): 0}

# each person has at most one confirmed match (allow less than for unsure matches)
for i in range(10):
    m1 += (lpSum(var[i]) <= 1)
for i in range(10):
    girl_matches = [var[j][i] for j in range(10)]
    m1 += (lpSum(girl_matches) <= 1)
    
# truth booth results
for k, v in truth_booths.items():
    m, f = k
    m1 += (var[m][f] == v)
    
# matching ceremony results
for r in rounds:
    matches = [var[m][f] for m, f in zip(range(10), r['pairs'])]
    m1 += (lpSum(matches) == r['num_matches'])
    
# Solve the optimization problem
status = m1.solve()

# Get the results
print('status: {}, {}'.format(m1.status, LpStatus[m1.status]))
print('objective: {}'.format(m1.objective.value()))
    
guys = ('Ethan', 'Ryan', 'Wes', 'Chris S', 'Chris T', 'Dre', 'JJ', 'Adam', 'Dillan', 'Joey')
girls = ('Jessica', 'Brittany', 'Simone', 'Shanley', 'Jacy', 'Amber', 'Paige', 'Kayla', 'Ashleigh', 'Coleysia')
results_table = []
for v in var:
    results_table.append([int(v[i].value()) for i in range(10)])
results_pd = pd.DataFrame(results_table, index=guys, columns=girls)
display(results_pd)

status: 1, Optimal
objective: 9.0


,Jessica,Brittany,Simone,Shanley,Jacy,Amber,Paige,Kayla,Ashleigh,Coleysia
Ethan,0,0,0,0,0,1,0,0,0,0
Ryan,1,0,0,0,0,0,0,0,0,0
Wes,0,0,0,0,0,0,0,1,0,0
Chris S,0,0,0,0,1,0,0,0,0,0
Chris T,0,0,0,0,0,0,1,0,0,0
Dre,0,0,1,0,0,0,0,0,0,0
JJ,0,0,0,0,0,0,0,0,0,0
Adam,0,0,0,1,0,0,0,0,0,0
Dillan,0,0,0,0,0,0,0,0,0,1
Joey,0,1,0,0,0,0,0,0,0,0


### Season 2

In [29]:
# Create the model
m2 = LpProblem(name='Are_You_The_One', sense=LpMinimize)

# Define decision variables
var = [[LpVariable(name='{},{}'.format(i,j), lowBound=0, cat='Binary') for j in range(11)] for i in range(10)]
var_flat = [x for tup in var for x in tup]

# Set objective function
m2 += lpSum(var_flat)

# Define constraints - results of each round
# In rounds: index is guys, values is girls
# In truth_booths: key is (guy, girl), value is result
rounds = []
rounds.append( {'pairs': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'num_matches': 2} )  # round 1
rounds.append( {'pairs': [7, 8, 2, 3, 1, 10, 4, 6, 0, 5], 'num_matches': 2} ) # round 2
rounds.append( {'pairs': [9, 1, 2, 3, 5, 8, 6, 0, 7, 4], 'num_matches': 3} )  # round 3
rounds.append( {'pairs': [7, 8, 2, 3, 1, 5, 4, 0, 6, 10], 'num_matches': 1} ) # round 4
rounds.append( {'pairs': [3, 6, 2, 8, 0, 4, 10, 1, 7, 5], 'num_matches': 3} ) # round 5
rounds.append( {'pairs': [0, 3, 2, 8, 6, 7, 10, 9, 4, 7], 'num_matches': 2} ) # round 6
rounds.append( {'pairs': [9, 1, 2, 8, 0, 10, 7, 3, 5, 4], 'num_matches': 6} ) # round 7
rounds.append( {'pairs': [9, 1, 2, 8, 0, 10, 6, 7, 3, 4], 'num_matches': 5} ) # round 8
rounds.append( {'pairs': [1, 10, 2, 8, 0, 3, 7, 6, 9, 4], 'num_matches': 8} ) # round 9
truth_booths = {(5,7): 0, (5,5): 0, (5,6): 0, (2,2): 1, (2,5): 0, 
                (3,8): 1, (3,5): 0, (4,4): 0, (4,0): 1, (8,5): 0,
                (9,4): 1, (1,1): 1, (1,5): 0}

# each person has at most one confirmed match (allow less than for unsure matches)
for i in range(10):
    m2 += (lpSum(var[i]) <= 1)
for i in range(11):
    girl_matches = [var[j][i] for j in range(10)]
    m2 += (lpSum(girl_matches) <= 1)
    
# truth booth results
for k, v in truth_booths.items():
    m, f = k
    m2 += (var[m][f] == v)
    
# matching ceremony results
for r in rounds:
    matches = [var[m][f] for m, f in zip(range(10), r['pairs'])]
    m2 += (lpSum(matches) == r['num_matches'])
    
# Solve the optimization problem
status = m2.solve()

# Get the results
print('status: {}, {}'.format(m2.status, LpStatus[m2.status]))
print('objective: {}'.format(m2.objective.value()))
    
guys = ('Layton', 'Dario', 'Pratt', 'Curtis', 'John', 'Brandon', 'Garland', 'Anthony', 'Nathan', 'Alex')
girls = ('Jenni', 'Ashley', 'Paris', 'Brianna', 'Jasmine', 'Christina', 'Alexandria', 'Jessica', 'Shelby', 'Ellie', 'Tyler')
results_table = []
for v in var:
    results_table.append([int(v[i].value()) for i in range(11)])
results_pd = pd.DataFrame(results_table, index=guys, columns=girls)
display(results_pd)

status: 1, Optimal
objective: 9.0


,Jenni,Ashley,Paris,Brianna,Jasmine,Christina,Alexandria,Jessica,Shelby,Ellie,Tyler
Layton,0,0,0,0,0,0,0,0,0,0,0
Dario,0,1,0,0,0,0,0,0,0,0,0
Pratt,0,0,1,0,0,0,0,0,0,0,0
Curtis,0,0,0,0,0,0,0,0,1,0,0
John,1,0,0,0,0,0,0,0,0,0,0
Brandon,0,0,0,1,0,0,0,0,0,0,0
Garland,0,0,0,0,0,0,0,1,0,0,0
Anthony,0,0,0,0,0,0,1,0,0,0,0
Nathan,0,0,0,0,0,0,0,0,0,1,0
Alex,0,0,0,0,1,0,0,0,0,0,0


In [ ]:
import cvxpy # to be experimented with

In [ ]:
from ortools.linear_solver import pywraplp # to be experimented with